In [1]:
import os
from zenml.repo import Repository
from zenml.datasources import CSVDatasource
from zenml.pipelines import TrainingPipeline
from zenml.steps.evaluator import TFMAEvaluator
from zenml.steps.preprocesser import StandardPreprocesser
from zenml.steps.split import RandomSplit
from zenml.steps.trainer import TFFeedForwardTrainer
from zenml.repo import Repository, ArtifactStore
from zenml.utils.naming_utils import transformed_label_name
from zenml.steps.deployer import GCAIPDeployer
from zenml.steps.deployer import CortexDeployer
from examples.cortex.predictor.tf import TensorFlowPredictor
from zenml.backends.orchestrator import OrchestratorGCPBackend
from zenml.metadata import MySQLMetadataStore
from zenml.backends.processing import ProcessingDataFlowBackend
from zenml.backends.training import SingleGPUTrainingGCAIPBackend

# Set up some variables

In [3]:
GCP_BUCKET=os.getenv('GCP_BUCKET')
GCP_PROJECT=os.getenv('GCP_PROJECT')
GCP_REGION=os.getenv('GCP_REGION')
GCP_CLOUD_SQL_INSTANCE_NAME=os.getenv('GCP_CLOUD_SQL_INSTANCE_NAME')
MODEL_NAME=os.getenv('MODEL_NAME')
CORTEX_ENV=os.getenv('CORTEX_ENV')
MYSQL_DB=os.getenv('MYSQL_DB')
MYSQL_USER=os.getenv('MYSQL_USER')
MYSQL_PWD=os.getenv('MYSQL_PWD')
MYSQL_PORT=os.getenv('MYSQL_PORT')
MYSQL_HOST=os.getenv('MYSQL_HOST')
CONNECTION_NAME = f'{GCP_PROJECT}:{GCP_REGION}:{GCP_CLOUD_SQL_INSTANCE_NAME}'
TRAINING_JOB_DIR = os.path.join(GCP_BUCKET, 'gcp_gcaip_training/staging')

In [4]:
GCP_BUCKET='gs://zenmlartifactstore'  # to be used as the artifact store
GCP_PROJECT ='core-engine'
GCP_REGION='europe-west1';
GCP_CLOUD_SQL_INSTANCE_NAME='mlmetadata'
MODEL_NAME='demomodel2'
CORTEX_ENV = 'gcp'
MYSQL_DB='vm_orchestrated'
MYSQL_USER='mlmetadata'
MYSQL_PWD='JjIwd3u7JbtveBgu'
MYSQL_PORT=3306
MYSQL_HOST='127.0.0.1'
CONNECTION_NAME = f'{GCP_PROJECT}:{GCP_REGION}:{GCP_CLOUD_SQL_INSTANCE_NAME}'
TRAINING_JOB_DIR = os.path.join(GCP_BUCKET, 'gcp_gcaip_training/staging')

In [5]:
repo: Repository = Repository.get_instance()
    
artifact_store = ArtifactStore(os.path.join(GCP_BUCKET, 'all_feature_demo'))

# Create first pipeline

In [6]:
training_pipeline = TrainingPipeline(name='Experiment 1')

2021-04-27 10:46:30,928 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 1 created.


#### Add a datasource. This will automatically track and version it.

In [7]:
try:
    ds = CSVDatasource(name='Pima Indians Diabetes', path='gs://zenml_quickstart/diabetes.csv')
except:
    repo: Repository = Repository.get_instance()
    ds = repo.get_datasource_by_name('Pima Indians Diabetes')
training_pipeline.add_datasource(ds)

2021-04-27 10:46:32,371 — zenml.datasources.base_datasource — INFO — Datasource Pima Indians Diabetes created.


#### Add a split step to partition data into train and eval

In [8]:
training_pipeline.add_split(RandomSplit(split_map={'train': 0.7, 'eval': 0.2, 'test':0.1}))

#### Add a preprocessing step to transform data to be ML-capable

In [9]:
training_pipeline.add_preprocesser(
    StandardPreprocesser(
        features=['times_pregnant', 'pgc', 'dbp', 'tst', 'insulin', 'bmi',
                  'pedigree', 'age'],
        labels=['has_diabetes'],
        overwrite={'has_diabetes': {
            'transform': [{'method': 'no_transform', 'parameters': {}}]}}
    ))

#### Add a trainer which defines model and training

In [10]:
training_pipeline.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=5))

#### Add an evaluator to calculate slicing metrics

In [11]:
training_pipeline.add_evaluator(
    TFMAEvaluator(slices=[['has_diabetes']],
                  metrics={transformed_label_name('has_diabetes'):
                     ['binary_crossentropy', 'binary_accuracy']}))

#### Run and evaluate

In [12]:
training_pipeline.run()

2021-04-27 10:46:38,771 — zenml.pipelines.training_pipeline — INFO — Datasource Pima Indians Diabetes has no commits. Creating the first one..
2021-04-27 10:46:38,774 — zenml.pipelines.base_pipeline — INFO — Pipeline 1619513198774 created.
2021-04-27 10:46:38,821 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is running.
2021-04-27 10:46:39,401 — zenml.datasources.csv_datasource — INFO — Matched 1: ['gs://zenml_quickstart/diabetes.csv']
2021-04-27 10:46:39,406 — zenml.datasources.csv_datasource — INFO — Using header from file: gs://zenml_quickstart/diabetes.csv.
2021-04-27 10:46:39,797 — zenml.datasources.csv_datasource — INFO — Header: ['times_pregnant', 'pgc', 'dbp', 'tst', 'insulin', 'bmi', 'pedigree', 'age', 'has_diabetes'].


2021-04-27 10:46:42,833 — apache_beam.internal.gcp.auth — WARNING — Unable to find default credentials to use: The Application Default Credentials are not available. They are available if running in Google Compute Engine. Otherwise, the environment variable GOOGLE_APPLICATION_CREDENTIALS must be defined pointing to a file defining the credentials. See https://developers.google.com/accounts/docs/application-default-credentials for more information.
Connecting anonymously.
2021-04-27 10:46:43,277 — apache_beam.runners.interactive.interactive_environment — WARNING — Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.


2021-04-27 10:46:45,237 — apache_beam.io.tfrecordio — WARNING — Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.


2021-04-27 10:46:45,473 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is finished.
2021-04-27 10:46:45,481 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is running.
2021-04-27 10:46:46,799 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is finished.
2021-04-27 10:46:46,802 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataSchema is running.


2021-04-27 10:46:46,843 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_util.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


...

      1/Unknown - 0s 137us/step - loss: 0.5849 - accuracy: 0.7500

2021-04-27 10:47:06,569 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2021-04-27 10:47:06,583 — tensorflow — WARNING — Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_train_batch_end` time: 0.0145s). Check your callbacks.


64/64 [==============================]0.6183 - accuracy: 0.6481  - 0s 2ms/step - loss: 0.6151 - accuracy: 0.676 - 0s 5ms/step - loss: 0.6124 - accuracy: 0.6797 - val_loss: 0.6636 - val_accuracy: 0.6726
Epoch 2/5
64/64 [==============================] - ETA: 0s - loss: 0.4887 - accuracy: 0.62 - ETA: 0s - loss: 0.5813 - accuracy: 0.71 - 0s 2ms/step - loss: 0.5849 - accuracy: 0.7090 - val_loss: 0.6420 - val_accuracy: 0.7083
Epoch 3/5
64/64 [==============================] - ETA: 0s - loss: 0.4069 - accuracy: 1.00 - ETA: 0s - loss: 0.5569 - accuracy: 0.73 - 0s 3ms/step - loss: 0.5682 - accuracy: 0.7207 - val_loss: 0.6189 - val_accuracy: 0.7202
Epoch 4/5
64/64 [==============================] - ETA: 0s - loss: 0.5433 - accuracy: 0.62 - ETA: 0s - loss: 0.5259 - accuracy: 0.76 - 0s 3ms/step - loss: 0.5430 - accuracy: 0.7461 - val_loss: 0.5854 - val_accuracy: 0.7202
Epoch 5/5
64/64 [==============================] - ETA: 0s - loss: 0.6116 - accuracy: 0.75 - ETA: 0s - loss: 0.5374 - accuracy: 0

/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['insulin', 'pedigree', 'has_diabetes', 'times_pregnant', 'pgc', 'age', 'dbp', 'has_diabetes_xl', 'bmi', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['age', 'bmi', 'dbp', 'has_diabetes', 'has_diabetes_xl', 'insulin', 'pedigree', 'pgc', 'times_pregnant', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
2021-04-27 10:47:11,246 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.

2021-04-27 10:47:11,798 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Trainer is finished.
2021-04-27 10:47:11,802 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is running.
2021-04-27 10:47:16,256 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is finished.


In [ ]:
training_pipeline.view_statistics(magic=True)

In [ ]:
training_pipeline.evaluate(magic=True)

#### Inspect datasource

In [ ]:
datasources = repo.get_datasources()
datasource = datasources[0]
print(datasource)

In [ ]:
df = datasource.sample_data()
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

## Skip preprocessing with your next (warm-starting) pipeline

#### Clone first experiment and only change one hyper-parameter

In [ ]:
training_pipeline_2 = training_pipeline.copy('Experiment 2')
training_pipeline_2.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=20))

In [ ]:
training_pipeline_2.run()

In [ ]:
training_pipeline_2.evaluate(magic=True)

## Post-training

#### Verify theres still only one datasource

In [ ]:
datasources = repo.get_datasources()
print(f"We have {len(datasources)} datasources")

#### Compare pipelines

In [ ]:
repo.compare_training_runs()

# Easily train on the cloud

In [ ]:
training_pipeline_3 = training_pipeline.copy('Experiment 3')

# Add a trainer with a GCAIP backend
training_backend = SingleGPUTrainingGCAIPBackend(
    project=GCP_PROJECT,
    job_dir=TRAINING_JOB_DIR
)

training_pipeline.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=20).with_backend(training_backend))

training_pipeline_3.run(artifact_store=artifact_store)

# Orchestrate every step on the Cloud

In [ ]:
training_pipeline_4 = training_pipeline.copy('Experiment 4')

# Define the metadata store
metadata_store = MySQLMetadataStore(
    host=MYSQL_HOST,
    port=int(MYSQL_PORT),
    database=MYSQL_DB,
    username=MYSQL_USER,
    password=MYSQL_PWD,
)


# Define the orchestrator backend
orchestrator_backend = OrchestratorGCPBackend(
    cloudsql_connection_name=CONNECTION_NAME,
    project=GCP_PROJECT,
    preemptible=True,  # reduce costs by using preemptible instances
    machine_type='n1-standard-4',
    gpu='nvidia-tesla-k80',
    gpu_count=1,
)


# Run the pipeline
training_pipeline_4.run(
    backend=orchestrator_backend,
    metadata_store=metadata_store,
    artifact_store=artifact_store,
)

# Add a deployer step with different integrations

## Option 1: Deploy to Google Cloud AI Platform

In [ ]:
training_pipeline_5 = training_pipeline.copy('Experiment 5')
training_pipeline_5.add_deployment(
    GCAIPDeployer(
        project_id=GCP_PROJECT,
        model_name=MODEL_NAME,
    )
)

training_pipeline_5.run(artifact_store=artifact_store)

## Option 2: Deploy to Kubernetes via Cortex

In [ ]:
training_pipeline = repo.get_pipeline_by_name('Experiment 1')

In [ ]:
training_pipeline_6 = training_pipeline.copy('Experiment 7')

# Add cortex deployer
api_config = {
    "name": MODEL_NAME,
    "kind": "RealtimeAPI",
    "predictor": {
        "type": "tensorflow",
        "models": {"signature_key": "serving_default"}}
}
training_pipeline.add_deployment(
    CortexDeployer(
        env=CORTEX_ENV,
        api_config=api_config,
        predictor=TensorFlowPredictor,
    )
)

training_pipeline_6.run(artifact_store=artifact_store)